In [1]:
import pandas as pd


import re
import nltk
from gensim.parsing.preprocessing import remove_stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
nltk.download('punkt')


[nltk_data] Downloading package punkt to /home/tim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
'''
def remove_date(text):
    return re.sub(r'\d{4}\-\d{1,2}\-\d{1,2}', ' ', text)

def remove_special_characters(text):
    text = re.sub(r'[^a-zA-Z0-9-]', ' ', text)
    ## Remove '-' but keep negative  (i.e. 'sign-in' => 'sign in', '------', => '' , '-123' => '-123') 
    text = re.sub(r'(\s+)(\-)(\d+)', r'\1#\3', text)
    text = re.sub(r'\-',' ', text)
    text = re.sub(r'#', '-', text)
    return text

def remove_multiple_spaces(text):
    return re.sub('\s+', ' ', text).strip()

def process_text_into_words(text):
    text = remove_date(text)
    text = remove_special_characters(text)
    text = remove_multiple_spaces(text.lower())
    text = remove_stopwords(text)
    words = word_tokenize(text)
    return words

def process_text_into_word_lists(text):
    sentences = text.split('\n')
    output_sentence = list(map(process_text_into_words, sentences))
    output_sentence = list(filter(None, output_sentence))
    return output_sentence


READ_PROPORTION_FOR_NOTES = 30
notes = pd.read_csv('NOTEEVENTS.csv.gz',skiprows = [i for i in range(1, 2083180*(READ_PROPORTION_FOR_NOTES-1) // READ_PROPORTION_FOR_NOTES) ], compression='gzip') #only 1619465 has charttime
notes.columns = notes.columns.str.lower()

import itertools
notes['text'] = notes['text'].map(process_text_into_word_lists)
notes['text_bert'] = notes['text'].map(lambda sentences: [' '.join(sentence) for sentence in sentences])
notes['text_tfidf']= notes['text'].map(lambda sentences: list(itertools.chain(*sentences)))
notes = notes.drop(columns=['text']).reset_index(drop=True)
'''

"\ndef remove_date(text):\n    return re.sub(r'\\d{4}\\-\\d{1,2}\\-\\d{1,2}', ' ', text)\n\ndef remove_special_characters(text):\n    text = re.sub(r'[^a-zA-Z0-9-]', ' ', text)\n    ## Remove '-' but keep negative  (i.e. 'sign-in' => 'sign in', '------', => '' , '-123' => '-123') \n    text = re.sub(r'(\\s+)(\\-)(\\d+)', r'\x01#\x03', text)\n    text = re.sub(r'\\-',' ', text)\n    text = re.sub(r'#', '-', text)\n    return text\n\ndef remove_multiple_spaces(text):\n    return re.sub('\\s+', ' ', text).strip()\n\ndef process_text_into_words(text):\n    text = remove_date(text)\n    text = remove_special_characters(text)\n    text = remove_multiple_spaces(text.lower())\n    text = remove_stopwords(text)\n    words = word_tokenize(text)\n    return words\n\ndef process_text_into_word_lists(text):\n    sentences = text.split('\n')\n    output_sentence = list(map(process_text_into_words, sentences))\n    output_sentence = list(filter(None, output_sentence))\n    return output_sentence\n\n\

In [14]:
READ_PROPORTION_FOR_NOTES = 30
print(2083180*(READ_PROPORTION_FOR_NOTES-1) // READ_PROPORTION_FOR_NOTES)

2013740


In [3]:
notes = pd.read_pickle('cleaned_text.pkl')
notes = notes[notes['text_bert'].map(lambda d: len(d)) > 0]
notes = notes[notes['text_tfidf'].map(lambda d: len(d)) > 0]

In [4]:
# !pip3 install transformers # => need to pip install at the first time
import torch
from transformers import AutoTokenizer, AutoModel, AutoConfig

config = AutoConfig.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
config.update({'output_hidden_states':True})  ## Because we only want the last 4 hidden layers
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", config=config)
old_model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
## We use only last 4 layers in the BERT model as mentioned in the paper
import copy
import torch.nn as nn
def deleteEncodingLayers(model):  # must pass in the full bert model
    oldModuleList = model.encoder.layer
    newModuleList = nn.ModuleList()
    # Now iterate over all layers, only keepign only the relevant layers.
    for i in range(8, 12):
        newModuleList.append(oldModuleList[i])

    # create a copy of the model, modify it with the new list, and return
    copyOfModel = copy.deepcopy(model)
    copyOfModel.encoder.layer = newModuleList

    return copyOfModel

model = deleteEncodingLayers(old_model)

In [6]:
'''
## Mean count of tokens for each sentences is 10
def count_mean(sentence):
    tmp = [len(x.split()) for x in sentence]
    return (sum(tmp)/ len(tmp))

print((notes['text_bert'].map(count_mean)).mean())
'''

"\n## Mean count of tokens for each sentences is 10\ndef count_mean(sentence):\n    tmp = [len(x.split()) for x in sentence]\n    return (sum(tmp)/ len(tmp))\n\nprint((notes['text_bert'].map(count_mean)).mean())\n"

In [7]:
def get_document_wise_embedding(sentences):

    features = tokenizer(
        sentences,
        padding='max_length',
        max_length=10, ## use mean count of tokens
        truncation=True,
        return_tensors='pt', ## 'pt': Return PyTorch torch.Tensor objects
        return_attention_mask=True
    )

    mask = features['attention_mask']

    with torch.no_grad():
        outputs = model(**features)

    #all_hidden_states = outputs['last_hidden_state']

    #concatenate_pooling = torch.cat(
        #(all_hidden_states[-1], all_hidden_states[-2], all_hidden_states[-3], all_hidden_states[-4]), 0)
    #concatenate_mask = torch.cat(
        #(mask,mask,mask,mask), 0)

    concatenate_pooling = outputs['last_hidden_state']
    #concatenate_mask = mask
        
    #concatenate_mask = concatenate_mask.unsqueeze(-1)
    #mask_embeddings = concatenate_pooling * concatenate_mask.float()
    mask_embeddings = concatenate_pooling
    
    #mean_mask_embeddings = mask_embeddings.sum(dim=1)
    #for dim in range(concatenate_mask.size(0)):
    #    mean_mask_embeddings[dim] = mean_mask_embeddings[dim] / concatenate_mask[dim].sum()
    
    mean_mask_embeddings = mask_embeddings.mean(dim=1)
    document_wise_embedding = mean_mask_embeddings.mean(dim=0)
    
    return document_wise_embedding

REFERENCE:
https://www.kaggle.com/code/rhtsingh/utilizing-transformer-representations-efficiently/notebook
https://stackoverflow.com/questions/71434804/how-to-fed-last-4-concatenated-hidden-layers-of-bert-to-fc-layers
https://towardsdatascience.com/how-to-use-bert-from-the-hugging-face-transformer-library-d373a22b0209

In [8]:
%%time
notes['text_bert'] = notes['text_bert'].map(get_document_wise_embedding)
notes.to_pickle('text_bert.pkl')

CPU times: user 6h 3s, sys: 2min 5s, total: 6h 2min 8s
Wall time: 1h 30min 29s
